<a href="https://colab.research.google.com/github/diarrabell/fashion-recs/blob/main/fashion_recs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install git+https://github.com/diarrabell/fashion-recs.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/diarrabell/fashion-recs.git to /tmp/pip-req-build-amzrq01t
  Running command git clone --filter=blob:none --quiet https://github.com/diarrabell/fashion-recs.git /tmp/pip-req-build-amzrq01t
  Resolved https://github.com/diarrabell/fashion-recs.git to commit 439a0e95cdc7f8a47592f0e31a53014aa5394c1a
ERROR: git+https://github.com/diarrabell/fashion-recs.git does not appear to be a Python project: neither 'setup.py' nor 'pyproject.toml' found.
